In [16]:
import arcpy
import os

def diagnose_and_repair_geometries(input_fc, output_gdb, run_repair=True):
    """
    Diagnose and optionally repair invalid geometries from a feature class.

    Args:
        input_fc (str): Path to input feature class (e.g. .sde or .gdb feature class)
        output_gdb (str): Path to output file geodatabase (will be created if doesn't exist)
        run_repair (bool): Whether to attempt geometry repair using buffer(0)

    Returns:
        dict: Summary of diagnostics and (optional) repairs
    """
    arcpy.env.overwriteOutput = True

    # Create output GDB if it doesn't exist
    if not arcpy.Exists(output_gdb):
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    spatial_ref = arcpy.Describe(input_fc).spatialReference

    # Output feature class names
    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None

    # Create output schemas
    arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                        geometry_type="POLYGON", spatial_reference=spatial_ref)
    if run_repair:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

    invalid_count = 0
    repaired_count = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]


    with arcpy.da.SearchCursor(input_fc, fields) as search_cursor, \
         arcpy.da.InsertCursor(invalid_fc, ["SHAPE@"]) as invalid_writer, \
         (arcpy.da.InsertCursor(repaired_fc, ["SHAPE@"]) if run_repair else None) as repaired_writer:

        while True:
            try:
                row = next(search_cursor)
            except StopIteration:
                break
            except Exception as fetch_err:
                print(f"❌ Failed to read row from cursor: {fetch_err}")
                continue

            try:
                oid, shape, treefarm_id, parcelnumber = row

                if shape is None:
                    print(f"⚠️ Null shape at OID {oid}")
                    continue

                if not shape.isValid:
                    invalid_writer.insertRow((shape,))
                    invalid_count += 1

                    if run_repair:
                        try:
                            repaired_shape = shape.buffer(0)
                            if repaired_shape and repaired_shape.isValid:
                                repaired_writer.insertRow((repaired_shape,))
                                repaired_count += 1
                            else:
                                print(f"⚠️ Repaired still invalid (OID {oid})")
                        except Exception as repair_err:
                            print(f"❌ Repair failed at OID {oid}: {repair_err}")

            except Exception as row_process_err:
                print(f"❌ Failed to process geometry at row: {row_process_err}")


    summary = {
        "🔎 Total Invalid Geometries": invalid_count,
        "🛠️  Total Repaired Geometries": repaired_count if run_repair else "Not run",
        "📍 Invalid Geometries Output": invalid_fc,
        "🧽 Repaired Geometries Output": repaired_fc if run_repair else "Not run"
    }

    print("\n📋 Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    return summary

result = diagnose_and_repair_geometries(
    input_fc = r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
    output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
    run_repair=True
)



❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm][STATE_ID = 0]
❌ Failed to read row from cursor: The number of points is less than required for feature [atfs_gdb.dbo.TreeFarm

KeyboardInterrupt: 

In [ ]:
import arcpy
import os
import csv

unreadable_count = 0

def diagnose_and_repair_geometries(input_fc, output_gdb, run_repair=True):
    arcpy.env.overwriteOutput = True

    # Ensure output GDB exists
    if not arcpy.Exists(output_gdb):
        arcpy.CreateFileGDB_management(os.path.dirname(output_gdb), os.path.basename(output_gdb))

    spatial_ref = arcpy.Describe(input_fc).spatialReference

    invalid_fc = os.path.join(output_gdb, "InvalidGeometries")
    repaired_fc = os.path.join(output_gdb, "RepairedGeometries") if run_repair else None
    log_csv = os.path.join(output_gdb, "GeometryDiagnostics.csv")

    arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="InvalidGeometries",
                                        geometry_type="POLYGON", spatial_reference=spatial_ref)
    if run_repair:
        arcpy.CreateFeatureclass_management(out_path=output_gdb, out_name="RepairedGeometries",
                                            geometry_type="POLYGON", spatial_reference=spatial_ref)

    invalid_count = 0
    repaired_count = 0
    unreadable_count = 0

    fields = ["OID@", "SHAPE@", "treefarm_id", "parcelnumber"]



    with open(log_csv, mode="w", newline="") as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow(["OID", "TreeFarmID", "ParcelNumber", "Status", "Details"])

        with arcpy.da.SearchCursor(input_fc, fields) as search_cursor, \
             arcpy.da.InsertCursor(invalid_fc, ["SHAPE@"]) as invalid_writer, \
             (arcpy.da.InsertCursor(repaired_fc, ["SHAPE@"]) if run_repair else None) as repaired_writer:

            while True:
                try:
                    row = next(search_cursor)
                except StopIteration:
                    break
                except Exception as fetch_err:
                    unreadable_count += 1
                    # ❌ Log as unreadable with minimal info
                    logwriter.writerow(["?", "?", "?", "Unreadable", str(fetch_err)])
                    continue

                try:
                    oid, shape, treefarm_id, parcelnumber = row

                    if shape is None:
                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Null Geometry", "Shape is None"])
                        continue

                    if not shape.isValid:
                        invalid_writer.insertRow((shape,))
                        invalid_count += 1
                        logwriter.writerow([oid, treefarm_id, parcelnumber, "Invalid", "Failed isValid()"])

                        if run_repair:
                            try:
                                repaired_shape = shape.buffer(0)
                                if repaired_shape and repaired_shape.isValid:
                                    repaired_writer.insertRow((repaired_shape,))
                                    repaired_count += 1
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repaired", "Buffer(0) succeeded"])
                                else:
                                    logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Failed", "Still invalid"])
                            except Exception as repair_err:
                                logwriter.writerow([oid, treefarm_id, parcelnumber, "Repair Error", str(repair_err)])
                    else:
                        # Log valid if you want full audit
                        # logwriter.writerow([oid, treefarm_id, parcelnumber, "Valid", ""])
                        pass

                except Exception as row_process_err:
                    unreadable_count += 1
                    logwriter.writerow(["?", "?", "?", "Processing Error", str(row_process_err)])
                    continue





    summary = {
        "🔎 Invalid Geometries": invalid_count,
        "🧽 Repaired Geometries": repaired_count if run_repair else "Not run",
        "🚫 Unreadable Features": unreadable_count,
        "📍 Invalid Features Output": invalid_fc,
        "🚫 Unreadable Features Skipped": unreadable_count,
        "🛠️  Repaired Features Output": repaired_fc if run_repair else "Not run",
        "📝 Diagnostics CSV": log_csv
    }

    print("\n📋 Geometry Validation Summary:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    return summary

result = diagnose_and_repair_geometries(
    input_fc = r"C:\Mac\Home\Documents\ArcGIS\Projects\ATFS_GeomErrors\SQLServer-100-atfs_gdb(dbeaver).sde\atfs_gdb.dbo.TreeFarm",
    output_gdb=r"C:\temp\TreeFarm_Validation.gdb",
    run_repair=True
)
